<center><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks&para;">unSkript Runbooks<a class="jp-InternalAnchorLink" href="#unSkript-Runbooks" target="_self" rel="noopener">&para;</a><a class="jp-InternalAnchorLink" href="#unSkript-Runbooks&para;" target="_self">&para;</a></h1>
<div class="alert alert-block alert-success">
<h3 id="Objective&para;">Objective<a class="jp-InternalAnchorLink" href="#Objective" target="_self" rel="noopener">&para;</a><a class="jp-InternalAnchorLink" href="#Objective&para;" target="_self">&para;</a></h3>
<br><strong><em>Find and Delete unused AWS Secrets</em></strong></div>
</center>
<p>&nbsp;</p>
<center>
<h2 id="Delete-Unused-AWS-Secrets"><u>Delete Unused AWS Secrets</u><a class="jp-InternalAnchorLink" href="#Delete-Unused-AWS-Secrets" target="_self">&para;</a></h2>
</center>
<h1 id="Steps-Overview&para;">Steps Overview<a class="jp-InternalAnchorLink" href="#Steps-Overview" target="_self" rel="noopener">&para;</a><a class="jp-InternalAnchorLink" href="#Steps-Overview&para;" target="_self">&para;</a></h1>
<p>1.&nbsp; Find unused secrets<br>2.&nbsp;&nbsp;Delete unused secrets</p>

In [5]:
if secret_names and not region:
    raise SystemExit("Provide a region for the AWS Secret ID's!")
if region == None:
    region = ""

<h3 id="Filter-unused-Secrets"><a id="1" target="_self" rel="nofollow"></a>Filter unused Secrets<a class="jp-InternalAnchorLink" href="#Filter-unused-Secrets" target="_self">&para;</a></h3>
<p>Using unSkript's Filter AWS Unused Secrets action, we will find unused secrets given a threshold number of days from their last use date. By default threshold number of days is set to <strong><span style="text-decoration: underline;">30</span></strong>.</p>
<blockquote>
<p>This action takes the following parameters: <code>region, threhold_days</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>unused_secrets</code></p>
</blockquote>

In [ ]:
##
##  Copyright (c) 2023 unSkript, Inc
##  All rights reserved.
##
import pprint
from typing import Optional, Tuple
from datetime import datetime, timedelta
from pydantic import BaseModel, Field
from unskript.connectors.aws import aws_get_paginator
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions
import pytz


from beartype import beartype
@beartype
def aws_list_unused_secrets_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_list_unused_secrets(handle, region: str = "", max_age_days: int = 30) -> Tuple:
    """aws_list_unused_secrets Returns an array of unused secrets.

        :type region: string
        :param region: AWS region.

        :type max_age_days: int
        :param max_age_days: The threshold to check the last use of the secret.

        :rtype: Tuple with status result and list of unused secrets.
    """
    result = []
    all_regions = [region]
    if not region:
        all_regions = aws_list_all_regions(handle)

    for reg in all_regions:
        try:
            # Filtering the secrets by region
            ec2Client = handle.client('secretsmanager', region_name=reg)
            res = aws_get_paginator(ec2Client, "list_secrets", "SecretList")
            for secret in res:
                secret_dict = {}
                secret_id = secret['Name']
                last_accessed_date = ec2Client.describe_secret(SecretId=secret_id)
                if 'LastAccessedDate' in last_accessed_date:
                    if last_accessed_date["LastAccessedDate"] < datetime.now(pytz.UTC) - timedelta(days=int(max_age_days)):
                        secret_dict["secret_name"] = secret_id
                        secret_dict["region"] = reg
                        result.append(secret_dict)
                else:
                    if last_accessed_date["CreatedDate"] < datetime.now(pytz.UTC) - timedelta(days=int(max_age_days)):
                        secret_dict["secret_name"] = secret_id
                        secret_dict["region"] = reg
                        result.append(secret_dict)
        except Exception:
            pass

    if len(result) != 0:
        return (False, result)
    return (True, None)


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "region": "region",
    "max_age_days": "int(threshold_days)"
    }''')

task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not secret_names",
    "condition_result": true
    }''')

task.configure(outputName="unused_secrets")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_list_unused_secrets, lego_printer=aws_list_unused_secrets_printer, hdl=hdl, args=args)

<h3 id="Create-List-of-Unused-Secrets">Create List of Unused Secrets</h3>
<p>This action filters regions that have no unused secrets and creates a list of those that have them.</p>
<blockquote>
<p>This action takes the following parameters: <code>None</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>all_unused_secrets</code></p>
</blockquote>

In [7]:
all_unused_secrets = []
try:
    if unused_secrets[0] == False:
        for secret in unused_secrets[1]:
            all_unused_secrets.append(secret)
except Exception as e:
    if secret_names:
        for name in secret_names:
            data_dict = {}
            data_dict["region"] = region
            data_dict["secret_name"] = name
            all_unused_secrets.append(data_dict)
    else:
        raise Exception(e)

<h3 id="Delete-unused-Secrets"><a id="2" target="_self" rel="nofollow"></a>Delete unused Secrets<a class="jp-InternalAnchorLink" href="#Delete-unused-Secrets" target="_self">&para;</a></h3>
<p>This action deleted unused secrets found in Step 1.&nbsp;</p>
<blockquote>
<p>This action takes the following parameters: <code>region, secret_name</code></p>
</blockquote>

In [ ]:
##
# Copyright (c) 2023 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Dict
import pprint


from beartype import beartype
@beartype
def aws_delete_secret_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_delete_secret(handle, region: str, secret_name: str) -> Dict:
    """aws_delete_secret Dict with secret details.

        :type handle: object
        :param handle: Object returned from Task Validate

        :type secret_name: string
        :param secret_name: Name of the secret to be deleted.

        :type region: string
        :param region: AWS Region.

        :rtype: Dict with secret details.
    """
    try:
        secrets_client = handle.client('secretsmanager', region_name=region)
        response = secrets_client.delete_secret(SecretId=secret_name)
        return response
    except Exception as e:
        raise Exception(e)


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "region": "iter.get(\\"region\\")",
    "secret_name": "iter.get(\\"secret_name\\")"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "all_unused_secrets",
    "iter_parameter": ["region","secret_name"]
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "if len(all_unused_secrets)!=0",
    "condition_result": true
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_delete_secret, lego_printer=aws_delete_secret_printer, hdl=hdl, args=args)

<h3 id="Conclusion">Conclusion</h3>
<p>In this Runbook, we were able to filter unused secrets and delete those keys. To view the full platform capabilities of unSkript please visit&nbsp;<a href="https://us.app.unskript.io" target="_blank" rel="noopener">us.app.unskript.io</a></p>